In [1]:
from wq_api import *

session = sign_in(username,password)

<Response [201]>


In [2]:
# 所有参数
# 数据文件
index = 0

# multi-simulation单次测试数
sublist_size = 10

# 最大同时multi-simulation数(由于暂停时间不足，仍可能出现线程占满)
t_max = 10

# multi-simulation数占满时的休眠时间，较长的休眠时间有利于防止线程占满，但可能会导致同时测试线程过少
sleep_time = 400


In [3]:
import os

dir_path = './refine_data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

from_path = f'{dir_path}/{index}_refine_1.csv'
to_path = f'{dir_path}/{index}_refine_2.csv'

print(f'Next available file path: {to_path}')

Next available file path: ./refine_data/0_refine_2.csv


In [4]:
priority_alpha_df = pd.read_csv(from_path)
priority_alpha_df = priority_alpha_df.iloc[:, 1:]
priority_alpha_df['location'] = np.nan

In [5]:
# 遍历 DataFrame 的每一行
alpha_list_with_settings = []

for index, row in priority_alpha_df.iterrows():
    simulation_data = {
        'type': row['type'],
        'settings': {
            'instrumentType': row['instrumentType'],
            'region': row['region'],
            'universe': row['universe'],
            'delay': row['delay'],
            'decay': row['decay'],
            'neutralization': row['neutralization'],
            'truncation': row['truncation'],
            'pasteurization': row['pasteurization'],
            'unitHandling': row['unitHandling'],
            'nanHandling': row['nanHandling'],
            'language': row['language'],
            'visualization': row['visualization'],
        },
        'regular': row['regular']
    }
    alpha_list_with_settings.append(simulation_data)

In [6]:
import time
from time import sleep

# 记录开始时间
start_time = time.time()

t = 0
total_length = len(alpha_list_with_settings)
for i in range(0, total_length, sublist_size):
    sublist = alpha_list_with_settings[i:i + sublist_size]
    t = t + 1
    while True:
        try:
            if t >= t_max:
                print(f"wait for {sleep_time}s")
                sleep(sleep_time)
                t = 0
            
            # 计算运行时间
            elapsed_time = time.time() - start_time

            # 如果运行时间超过4小时，重新登录
            if (elapsed_time > 4 * 3600):
                session = sign_in(username, password)
                start_time = time.time()  # 重置开始时间
                print("Session renewed successfully.")

            # 发送多重模拟请求
            response = session.post('https://api.worldquantbrain.com/simulations', json=sublist)

            # 检查响应
            if len(response.headers["Location"]) > 0:
                print("Alpha location get successfully.")
                for alpha_data in sublist:
                    priority_alpha_df.loc[priority_alpha_df['regular'] == alpha_data['regular'], 'location'] = response.headers["Location"]
                priority_alpha_df.to_csv(to_path)
                print(response.headers["Location"])
                break

        except Exception as e:
            # 打印错误信息
            print(f"Error in sending simulation request. Retrying after 5s. Error: {e}")
            sleep(5)

# 处理剩余的元素（如果有）
if total_length % sublist_size != 0:
    remaining_sublist = alpha_list_with_settings[-(total_length % sublist_size):]
    t = t + 1
    while True:
        try:
            if t >= t_max:
                print(f"wait for {sleep_time} s")
                sleep(sleep_time)
                t = 0

            # 计算运行时间
            elapsed_time = time.time() - start_time

            # 如果运行时间超过4小时，重新登录
            if (elapsed_time > 4 * 3600):
                session = sign_in(username, password)
                start_time = time.time()  # 重置开始时间
                print("Session renewed successfully.")

            # 发送多重模拟请求
            response = session.post('https://api.worldquantbrain.com/simulations', json=remaining_sublist)
            # 检查响应
            if len(response.headers["Location"]) > 0:
                print("Alpha location get successfully.")
                for alpha_data in remaining_sublist:
                    priority_alpha_df.loc[priority_alpha_df['regular'] == alpha_data['regular'], 'location'] = response.headers["Location"]
                priority_alpha_df.to_csv(to_path)
                print(response.headers["Location"])
                break

        except Exception as e:
            # 打印错误信息
            print(f"Error in sending simulation request. Retrying after 5s. Error: {e}")
            sleep(5)

Alpha location get successfully.
https://api.worldquantbrain.com/simulations/2InOuG3Je4Hkc551ciLbIjeD
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/1AgAiw22i55q8z1tme3NUim
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/4wk78j26n4hn9Ldu4CmhW7z
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/1IK3YV7G64yLarOSwfNbyJE
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/24mAM65pG4VVbkm1gCcJZMnJ
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/79v7MgB04x89NT1fgMfQfP2
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/2H8AEJdmH4LT8IGvgppCHOy
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/WT4W23p34l4b1x7FGNWqUm
Alpha location get successfully.
https://api.worldquantbrain.com/simulations/L6AWSbQx4ILa4114UmZqzif
wait for {sleep_time}s
Alpha location get successfully.
https://api.worldquantbrain.com/si